1. 라이브러리 설치

In [7]:
!pip install -q "transformers==4.44.2" "timm" "sentencepiece" "accelerate" "protobuf"


2. 데이터 파일 불러오기

In [8]:
from google.colab import files
import os

# 1) 로컬에서 donut_dataset_all.zip 업로드
uploaded = files.upload()  # 파일 선택 창에서 donut_dataset_all.zip 선택

# 2) 압축 풀기 (zip 파일명이 다르면 아래 이름 수정)
zip_name = [name for name in uploaded.keys()][0]  # 업로드한 첫 파일 이름
print("압축 파일:", zip_name)

!unzip -q {zip_name} -d .

# 구조 확인
print("현재 디렉토리:", os.listdir("."))
print("donut_dataset 내용:", os.listdir("donut_dataset"))


Saving donut_dataset.zip to donut_dataset.zip
압축 파일: donut_dataset.zip
현재 디렉토리: ['.config', 'donut_dataset.zip', 'donut_dataset', 'sample_data']
donut_dataset 내용: ['images_aug', 'labels.json', 'images']


3. Donut 모델 로드 (GPU 설정 포함)

In [9]:
import torch
from transformers import DonutProcessor, VisionEncoderDecoderModel

MODEL_NAME = "naver-clova-ix/donut-base-finetuned-cord-v2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 디바이스:", device)

processor = DonutProcessor.from_pretrained(MODEL_NAME)
model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME)

# pad 토큰 설정
if processor.tokenizer.pad_token_id is None:
    processor.tokenizer.pad_token = processor.tokenizer.eos_token
model.config.pad_token_id = processor.tokenizer.pad_token_id

# decoder 시작 토큰 설정 (없으면 <s> 사용)
if model.config.decoder_start_token_id is None:
    model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids("<s>")

# eos 토큰 설정
model.config.eos_token_id = processor.tokenizer.eos_token_id

# 메모리 절약 옵션
model.config.use_cache = False
model.gradient_checkpointing_enable()

model.to(device)

print("모델 / 프로세서 로드 완료")
print("pad_token_id:", model.config.pad_token_id)
print("decoder_start_token_id:", model.config.decoder_start_token_id)


사용 디바이스: cuda
모델 / 프로세서 로드 완료
pad_token_id: 1
decoder_start_token_id: 0


4. Dataset & DataLoader (원본 + 증강 180장 사용)

In [10]:
import os
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch

MAX_LEN = 80  # 라벨이 짧으니까 80 정도면 충분

class DonutReceiptDataset(Dataset):
    def __init__(self, image_dir_orig, image_dir_aug, label_path, processor, max_length=MAX_LEN):
        self.image_dir_orig = image_dir_orig
        self.image_dir_aug = image_dir_aug
        self.processor = processor
        self.max_length = max_length

        # 1) labels.json 읽기 (원본 파일명 기준)
        with open(label_path, "r", encoding="utf-8") as f:
            self.labels = json.load(f)

        self.image_files = []

        # (1) 원본 이미지: labels.json 에 있는 파일만 사용
        for fname in os.listdir(self.image_dir_orig):
            if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
                continue
            if fname in self.labels:
                self.image_files.append(fname)
            else:
                print(f"⚠ labels.json에 없는 원본 이미지: {fname}")

        # (2) 증강 이미지: 이름에서 원본 이름을 추출해서 라벨 매칭
        # 예: aug1_20251113_알촌_8300.jpg -> 20251113_알촌_8300.jpg
        for fname in os.listdir(self.image_dir_aug):
            if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            parts = fname.split("_", 1)
            if len(parts) < 2:
                print(f"⚠ 증강 파일 이름 형식 이상: {fname}")
                continue

            original_name = parts[1]
            if original_name in self.labels:
                self.image_files.append(fname)
            else:
                print(f"⚠ labels.json에 없는 증강 이미지(원본 라벨 없음): {fname} -> {original_name}")

        print(f"✅ Dataset 준비 완료. 총 이미지 개수: {len(self.image_files)}")

    def _get_image_path(self, file_name):
        aug_path = os.path.join(self.image_dir_aug, file_name)
        if os.path.exists(aug_path):
            return aug_path

        orig_path = os.path.join(self.image_dir_orig, file_name)
        if os.path.exists(orig_path):
            return orig_path

        raise FileNotFoundError(f"이미지 파일을 찾을 수 없습니다: {file_name}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        file_name = self.image_files[idx]
        img_path = self._get_image_path(file_name)

        # 1) 이미지 로드
        image = Image.open(img_path).convert("RGB")

        # 2) 라벨 텍스트 (labels.json 안에 있는 JSON 문자열 그대로)
        if file_name in self.labels:
            text = self.labels[file_name]
        else:
            # 증강 파일인 경우: augX_원본이름 -> 원본 이름으로 라벨 찾기
            parts = file_name.split("_", 1)
            original_name = parts[1] if len(parts) > 1 else file_name
            text = self.labels[original_name]

        # 3) 이미지 인코딩 → pixel_values
        image_enc = self.processor(
            image,
            return_tensors="pt"
        )
        pixel_values = image_enc.pixel_values.squeeze(0)   # (3, H, W)

        # 4) 텍스트 인코딩 → input_ids
        text_enc = self.processor.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        input_ids = text_enc.input_ids.squeeze(0)          # (max_length,)

        # 5) pad 토큰은 loss에서 무시되도록 -100 처리
        labels = input_ids.clone()
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        return {
            "pixel_values": pixel_values,
            "labels": labels,
        }


def collate_fn(batch):
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    labels = torch.stack([item["labels"] for item in batch])
    return {"pixel_values": pixel_values, "labels": labels}


dataset = DonutReceiptDataset(
    image_dir_orig="donut_dataset/images",
    image_dir_aug="donut_dataset/images_aug",
    label_path="donut_dataset/labels.json",
    processor=processor,
    max_length=MAX_LEN,
)

train_loader = DataLoader(
    dataset,
    batch_size=1,       # 메모리 안전하게 1 유지
    shuffle=True,
    collate_fn=collate_fn,
)

print("최종 데이터 개수:", len(dataset))


✅ Dataset 준비 완료. 총 이미지 개수: 180
최종 데이터 개수: 180


5. GPU + AMP 학습 루프

In [11]:
from torch.optim import AdamW
from tqdm.auto import tqdm
from torch.cuda.amp import autocast, GradScaler

optimizer = AdamW(model.parameters(), lr=3e-5)
scaler = GradScaler()

num_epochs = 4   # 4~5 정도 추천 (원하면 조절 가능)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch in pbar:
        pixel_values = batch["pixel_values"].to(device, non_blocking=True)
        labels = batch["labels"].to(device, non_blocking=True)

        optimizer.zero_grad()

        # 🔹 혼합 정밀도 (fp16)로 forward → 메모리 절약 + 속도 ↑
        with autocast():
            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        pbar.set_postfix({"loss": float(loss.item())})

    avg_loss = running_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} 평균 loss: {avg_loss:.4f}")

    if device.type == "cuda":
        torch.cuda.empty_cache()


/tmp/ipython-input-3817907376.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Epoch 1/4:   0%|          | 0/180 [00:00<?, ?it/s]

/tmp/ipython-input-3817907376.py:22: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
`use_cache=True` is incompatible with gradient checkpointing`. Setting `use_cache=False`...


✅ Epoch 1 평균 loss: 1.0725


Epoch 2/4:   0%|          | 0/180 [00:00<?, ?it/s]

✅ Epoch 2 평균 loss: 0.0814


Epoch 3/4:   0%|          | 0/180 [00:00<?, ?it/s]

✅ Epoch 3 평균 loss: 0.0652


Epoch 4/4:   0%|          | 0/180 [00:00<?, ?it/s]

✅ Epoch 4 평균 loss: 0.0504


6. 학습된 모델 저장

In [15]:
SAVE_DIR = "donut_finetuned"

processor.save_pretrained(SAVE_DIR)
model.save_pretrained(SAVE_DIR)

import shutil
shutil.make_archive("donut_finetuned", "zip", SAVE_DIR)


'/content/donut_finetuned.zip'

7. 저장한 모델 다운로드

In [16]:
from google.colab import files
files.download("donut_finetuned.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

8. 훈련 결과 간단 테스트

In [14]:
import json
from PIL import Image

# 테스트는 CPU로 돌려도 됨
test_device = torch.device("cpu")
model_test = VisionEncoderDecoderModel.from_pretrained(SAVE_DIR).to(test_device)
processor_test = DonutProcessor.from_pretrained(SAVE_DIR)
model_test.eval()

test_image_path = "donut_dataset/images/20251113_알촌_8300.jpg"  # 실제 존재하는 파일명으로 바꾸기

image = Image.open(test_image_path).convert("RGB")
pixel_values = processor_test(image, return_tensors="pt").pixel_values.to(test_device)

prompt = "<s>"
decoder_input_ids = processor_test.tokenizer(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
).input_ids.to(test_device)

with torch.no_grad():
    outputs = model_test.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=128,
        eos_token_id=processor_test.tokenizer.eos_token_id,
        pad_token_id=processor_test.tokenizer.pad_token_id,
    )

generated = processor_test.batch_decode(outputs, skip_special_tokens=True)[0]
print("RAW OUTPUT:", repr(generated))

start = generated.find("{")
end = generated.rfind("}")
if start != -1 and end != -1:
    json_str = generated[start:end+1]
    print("JSON TEXT:", json_str)
    print("파싱 결과:", json.loads(json_str))
else:
    print("JSON을 찾지 못함")


RAW OUTPUT: '{"date":"2025-11-13","store":"알촌","total":"8300"}'
JSON TEXT: {"date":"2025-11-13","store":"알촌","total":"8300"}
파싱 결과: {'date': '2025-11-13', 'store': '알촌', 'total': '8300'}
